In [4]:
import os
import matplotlib
import numpy as np
import open3d as o3d
import torch
from lib.utils.image_utils import color_mapping

In [8]:
def process_pcd(pcd_path, colormap=matplotlib.colormaps["rainbow"], centroid=None):
    pcd = o3d.io.read_point_cloud(pcd_path)
    
    # 将点云数据转换为PyTorch张量
    points = torch.from_numpy(np.asarray(pcd.points))
    points = points[torch.norm(points, dim=1) > 1e-6]

    if centroid is None:
        # 计算点云重心
        centroid = torch.mean(points, dim=0)
    
    # 将点云移动到重心位置
    points = points - centroid
    
    # 更新点云数据
    pcd.points = o3d.utility.Vector3dVector(points.numpy())
    points = torch.from_numpy(np.asarray(pcd.points))

    z = points[:, 2]
    max_q = torch.quantile(z, 0.9)
    min_q = torch.quantile(z, 0.1)
    z = (z - min_q) / (max_q - min_q)
    z = z.clamp(0, 1)

    color = color_mapping(z, colormap)
    pcd.colors = o3d.utility.Vector3dVector(color.cpu().numpy().astype(np.float64))

    return pcd, centroid

In [ ]:
pcd_gt_path = "output/default/test/scene_ks3/evals/30000/test/pointclouds/1947_gt_pcd.ply"
pcd_render_path = pcd_gt_path.replace("_gt_pcd.ply", "_rendered_pcd.ply")

pcd_gt, centroid = process_pcd(pcd_gt_path)
pcd_render, centroid = process_pcd(pcd_render_path, centroid=centroid, colormap=matplotlib.colormaps["viridis"])

output_path = "output/default/test/scene_ks3/evals/30000/test/pointclouds/output"
pcd_gt_save_path = os.path.join(output_path, pcd_gt_path.split("/")[-1])
pcd_render_save_path = os.path.join(output_path, pcd_render_path.split("/")[-1])

o3d.io.write_point_cloud(pcd_gt_save_path, pcd_gt)
o3d.io.write_point_cloud(pcd_render_save_path, pcd_render)


In [ ]:
list(matplotlib.colormaps.keys())